#Taller Final



*   Roberto José Ávila Moreno
*   Salim Abou-Ammar Garcia



**LIBRERIAS**

In [40]:
!pip install --quiet yfinance 
import yfinance as yf
import seaborn as sns
import pandas as pd 
import numpy as np 
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import linprog
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.simplefilter("ignore")

**INFORME:**

**Descripción del Problema**

Encontrar un plan de producción es uno de los procesos más difíciles y se debe recurrir a herramientas de optimización que permitan encontrar de manera efectiva y eficiente el plan que mejor se acomode a los objetivos. En el siguiente caso de estudio nos encontramos frente a un problema de una empresa manufacturera que tiene tres tipos de productos (A, B, C), también tiene tres instalaciones ubicadas en diferentes ciudades con capacidades de producción de 1000, 1200, y 1500, y tiene además que cumplir con una demanda de 500, 700, y 800, y con esto se decide cuanto producir de cada producto en las respectivas instalaciones de manera que se optimice el proceso.

**Justificación**

Se busca una solución al problema debido a que la empresa quiere o necesita minimizar los costos tanto de producción como de distribución, teniendo en cuenta principalmente la capacidad de producción de cada instalación, los costos de transporte de cada producto y la demanda de cada producto. Es importante la solución de este problema debido a que la rentabilidad de las empresas manufactureras dependen de la optimización de la producción y de lo relacionado con la parte operativa. Por lo que la clave de éxito de una empresa del sector es producir de la manera más efectiva posible.


**Objetivos**

* Trabajo: Poner en práctica todos los temas aprendidos a lo largo del curso para solucionar un problema aplicado a las finanzas, en este caso usaremos los aprendido en clase para solucionar un problema de optimización de producción de una empresa manufacturera.

* Caso: El objetivo principal de la empresa es lograr disminuir los costos supliendo con la cantidad óptima de bienes a producir, y teniendo en cuenta factores como la demanda, la capacidad de producción y los costos de inventario.

**DATOS DE ENTRADA**

In [41]:
#DATOS:
Productos=3
Instalaciones=3
Capacidades= np.array([1000,1200,1500])
Demandas= np.array([500,700,800])
Costos_por_instalacion=np.array([[5,8,9], [6,4,12], [9,5,10]])

**PRESENTACIÓN DE DATOS Y ANÁLISIS DESCRIPTIVO**

**Capacidad de Producción**

La empresa maneja 3 productos, el producto A, el producto B y el producto C. La empresa tiene tres instalaciones de producción ubicadas en diferentes ciudades con capacidades instaladas en cada una: 1000, 1200 y 1500, respectivamente; y tiene que decidir cuánto producir de cada producto en cada instalación si decide mantenerla abierta.

In [42]:
Tabla = {'Instalación':['1', '2', '3'],
         'Producto': ['A','B','C'], 
                   'Capacidad de Producción': ['1000','1200','1500']}

Tabla_Final = pd.DataFrame(Tabla,index=["1","2","3"])
Tabla_Final

,Instalación,Producto,Capacidad de Producción
1,1,A,1000
2,2,B,1200
3,3,C,1500


**Demanda**

La empresa debe saber cómo distribuir los tres productos para cubrir una restricción de la demanda que corresponde a 500, 700 y 800, respectivamente por producto.

In [43]:
Tabla = {'Productos':['A', 'B', 'C'], 
                   'Demanda a Satisfacer': ['500','700','800']}

Tabla_Final = pd.DataFrame(Tabla,index=["1","2","3"])
Tabla_Final

,Productos,Demanda a Satisfacer
1,A,500
2,B,700
3,C,800


**Costos de Distribución**

Y además de todo esto, también teniendo en cuenta los costos de distribución a cada punto de demanda.

In [44]:
Tabla = {'Instalación':['1', '2', '3'], 
                   'Punto de Demanda 1': ['$5','$6','$9'],
                   'Punto de Demanda 2':['$8','$4','$5'],
                   'Punto de Demanda 3':['$9','$12','$10']}

Tabla_Final = pd.DataFrame(Tabla,index=["1","2","3"])
Tabla_Final

,Instalación,Punto de Demanda 1,Punto de Demanda 2,Punto de Demanda 3
1,1,$5,$8,$9
2,2,$6,$4,$12
3,3,$9,$5,$10


**FUNCIÓN: OBJETIVO DE MINIMIZAR COSTOS**

**Variables**

* x = Cantidad del producto i producido en la instalación j

* y = Demanda del producto i

* c = Costo de transportar el producto i desde la instalación j hasta la ubicación de la demanda

* i = Productos
* j = Instalaciónes

**Restricciones**

* Capacidad de producción hasta de 1000, 1200, y 1500 por cada instalación 1, 2, y 3, respectivamente.
* Restricción de demanda minimo de 500, 700, y 800 por producto A, B, y C, respectivamente.
* Cantidades no negativas.

**Función**

Minimizar costos totales de producción y distribución

$$Min. Costo-Total$$


**OPTIMIZACIÓN**

Con el proceso de optimización lo que buscamos es encontrar las cantidades óptimas a producir de cada producto en cada instalación y además lograr ver cuál es la instalación menos efectiva, para de esta manera cerrarla.

In [45]:
# Variables de decisión
x = cp.Variable((Productos, Instalaciones))

# Función objetivo
Costo_t = cp.sum(cp.multiply(Costos_por_instalacion, x))

# Restricciones:

# Restricción de capacidad
for j in range(Instalaciones):
    cp.sum(x[:, j]) <= Capacidades[j]

# Restricción de demanda
for i in range(Productos):
    cp.sum(x[i, :]) == Demandas[i]

# Restricción de no negatividad
x >= 0

# Problema de optimización
prob = cp.Problem(cp.Minimize(Costo_t), [x >= 0, 
    cp.sum(x[:, 0]) <= Capacidades[0], 
    cp.sum(x[:, 1]) <= Capacidades[1],
    cp.sum(x[:, 2]) <= Capacidades[2],
    cp.sum(x[0, :]) == Demandas[0],
    cp.sum(x[1, :]) == Demandas[1],
    cp.sum(x[2, :]) == Demandas[2]])

# Solución del problema
prob.solve()

# Solución:

print("Costo total:", prob.value)
print("Cantidad de productos producidos:")
print(x.value)
pd.DataFrame(x.value).round(1)

Costo total: 9900.00002040023
Cantidad de productos producidos:
[[4.99999999e+02 2.92635419e-07 5.84871759e-07]
 [3.00000002e+02 3.99999998e+02 2.31390215e-07]
 [1.97877330e-06 7.99999997e+02 8.47020622e-07]]


,0,1,2
0,500.0,0.0,0.0
1,300.0,400.0,0.0
2,0.0,800.0,0.0


In [46]:
Tabla = {'Demanda Producto A': ['500','0','0'],
                   'Demanda Producto B':['300','400','0'],
                  'Demanda Producto C':['0','800','0']}

Tabla_Final = pd.DataFrame(Tabla,index=["Instalación 1","Instalación 2","Instalación 3"])
Tabla_Final

,Demanda Producto A,Demanda Producto B,Demanda Producto C
Instalación 1,500,300,0
Instalación 2,0,400,800
Instalación 3,0,0,0


Luego de aplicar la optimización, según los resultados, se debe cerrar la planta número 3, debido a los altos costos de distribución que dicha planta tiene, y de esta manera distribuir la producción de la siguiente manera:
La instalación 1 producirá 500 del producto A (demanda completa) y producirá 300 del producto B. Y, por otro lado, la instalación 2 producirá 400 del producto B (para completar demanda) y 800 del producto C (demanda completa).

**MONTE CARLO**

In [47]:
!pip install --quiet pyDOE
from pyDOE import lhs

Luego de tener la optimización con las demandas dadas, nos vemos en la necesidad de realizar simulaciones de demanda (con el método Monte Carlo) y optimizar dichas demandas simuladas una por una, para de esta manera ver el comportamiento del proceso de producción.

Dicho lo anterior, el proceso que realizamos fue realizar un ciclo for donde dentro de este aplicáramos una simulación para cada demanda (con Monte Carlo) y a cada simulación realizarle una optimización y que dicho proceso se repita dependiendo del número de optimizaciones y los resultados se vayan acumulando en una lista.

In [48]:
ns = 1000 #Numero de simulaciones

Simulación de Demandas

In [50]:
rands_u_D1 = lhs(1, ns) #Hyper Q Volatility
mu = 500 # Demanda 1
std = 25 #Desviacion de los datos (Digitado)
rands_n_D1 = norm(mu, std).ppf(rands_u_D1) #Distribución y Generacion de Datos Aleatorios
#plt.hist(rands)

In [51]:
rands_u_D2 = lhs(1, ns) #Hyper Q Volatility
mu = 700 # Demanda 2
std = 30 #Desviacion de los datos (Digitado)
rands_n_D2 = norm(mu, std).ppf(rands_u_D2) #Distribución y Generacion de Datos Aleatorios

In [52]:
rands_u_D3 = lhs(1, ns) #Hyper Q Volatility
mu = 800 #Demanda 3
std = 35 #Desviacion de los datos (Digitado)
rands_n_D3 = norm(mu, std).ppf(rands_u_D3) #Distribución y Generacion de Datos Aleatorios

Matriz de demandas simuladas

In [53]:
dem_m = np.concatenate((rands_n_D1.reshape(-1,1), rands_n_D2.reshape(-1,1), rands_n_D3.reshape(-1,1)), axis=1)
dem_m.shape #Creamos matriz de ns filas y 3 columnas (3 demandas)

(1000, 3)

Estructura seguida para realizar el ciclo for

In [54]:
#Estructura del Ciclo For
#Resultados = []

#for i in range(N de Simulaciones):
  #Montecarlo
  #Optimizacion
  #Resultados.append(Optimizacion)

#Resultados

In [55]:
#Resultados.mean()

Ciclo for:

In [60]:
Resultados = np.zeros(ns)
for k in range(ns):
  #Simulación
  Demanda_Sim = dem_m[k,:]

  #Variables de Decisión
  x = cp.Variable((Productos, Instalaciones))
  #Función Objetivo
  Costo_t = cp.sum(cp.multiply(Costos_por_instalacion, x))
  
  #Restricciones 

  for j in range(Instalaciones):
    cp.sum(x[:, j]) <= Capacidades[j]
  for i in range(Productos):
    cp.sum(x[i, :]) == Demanda_Sim[i]
  x >= 0
  
  #Problema de Optimizacion (Minimizar)
  prob = cp.Problem(cp.Minimize(Costo_t), [x >= 0, 
    cp.sum(x[:, 0]) <= Capacidades[0], 
    cp.sum(x[:, 1]) <= Capacidades[1],
    cp.sum(x[:, 2]) <= Capacidades[2],
    cp.sum(x[0, :]) == Demanda_Sim[0],
    cp.sum(x[1, :]) == Demanda_Sim[1],
    cp.sum(x[2, :]) == Demanda_Sim[2]])
  prob.solve()
  solucion = prob.value
  Resultados[k] = solucion

In [61]:
Resultados.mean()

9900.003424847993

Como vemos, el promedio de la optimización de las demandas simuladas nos da como resultado los mismos costos totales de producción de la optimización principal realizada anteriormente, por lo que de esta manera comprobamos que el ejercicio está bien realizado y los costos totales no se ven afectados por algún cambio en las demandas, y de esta manera se sigue con el mismo proceso de producción que se expuso anteriormente.

**CONCLUSIONES**

* ¿Cuál es la decisión óptima de la empresa en su plan de producción y de 
distribución?
* R/ Según los resultados presentados anteriormente, se debe cerrar la planta número 3, debido a los altos costos de distribución que dicha planta tiene, y de esta manera distribuir la producción de la siguiente manera:
La instalación 1 producirá 500 del producto A (demanda completa) y producirá 300 del producto B. Y, por otro lado, la instalación 2 producirá 400 del producto B (para completar demanda) y 800 del producto C (demanda completa).

* ¿Cambia la decisión del plan de producción de la empresa si se afectan las variables de demanda y de costos?
* R/ Según los resultados presentados anteriormente, los costos totales no se ven afectados por algún cambio en las demandas, y de esta manera se sigue con el mismo proceso de producción que se expuso anteriormente.